In [1]:
# Prophet model for time series forecast
import os
import darts
import prophet
import pyspark

# Data processing
import pandas as pd
import numpy as np
import csv

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Multi-processing
from multiprocessing import Pool, cpu_count

# Spark
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
from prophet import Prophet

# Process bar
from tqdm import tqdm

# Tracking time
from time import time

#hmm needed? 
import torch
import torch.nn as nn
import torch.optim as optim
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

#darts
# transformers and preprocessing
from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import TransformerModel, ExponentialSmoothing
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf


# Importing all functions in the "functions.py" file
#%run functions.py


%matplotlib inline
from tslearn.clustering import silhouette_score
from sklearn.decomposition import PCA
plt.rcParams['figure.figsize'] = [25, 8]

from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance


import matplotlib.pyplot as plt
from sklearn.cluster import BisectingKMeans
from datetime import datetime

/home/coder/.local/lib/python3.9/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [50]:
import glob
import numpy as np
%run functions.py

### Looping the cluster code over all cluster data sets


In [53]:
# Define the path to the folder
folder_path = '/work/Data-Science-Liv/clustered_data'

# Define the file pattern to match
file_pattern = 'long_*.csv'

# Get a list of file paths that match the pattern
file_paths = glob.glob(f"{folder_path}/{file_pattern}")

# Iterate over the file paths
for file_path in file_paths:
    df = pd.read_csv(file_path, parse_dates=['Year'])

    #Create test and train splits
    train_df, test_df = create_splits(df)

    #Perform yearly aggregation across all individual time series - in train_df
    agg_df_train = yr_aggregate_train(train_df)

    #define and fit model
    model = Prophet(
    growth='linear',
    seasonality_mode= 'additive',
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=False
    )
    model.fit(agg_df_train)
    
    # Generate forecasts on the test period for the median time series
    agg_df_test = yr_aggregate_test(test_df)
    predictions = model.predict(df=agg_df_test.drop('y', axis=1))

    #Rename to merge, merge, and concat data frames
    merged_train_df, merged_test_df, df_actual_all = rename_merge_concat(train_df, test_df, agg_df_train, agg_df_test, predictions)

    ##Get predictions for test period per country
    test_df_delta = df_actual_all.groupby(['Country']).apply(lambda x: get_linear_model_pred(x, agg_df_test)).reset_index(drop=False)
    test_df_delta = test_df_delta.rename(columns={'month': 'ds'})
    
    test_df_2 = pd.merge(df_actual_all, test_df_delta, on=['Country', 'ds'], how='left')
    test_df_2['final_prediction'] = test_df_2['yhat'] + test_df_2['test_delta_pred']





15:04:17 - cmdstanpy - INFO - Chain [1] start processing
15:04:17 - cmdstanpy - INFO - Chain [1] done processing
15:04:18 - cmdstanpy - INFO - Chain [1] start processing
15:04:18 - cmdstanpy - INFO - Chain [1] done processing
15:04:19 - cmdstanpy - INFO - Chain [1] start processing
15:04:19 - cmdstanpy - INFO - Chain [1] done processing
15:04:20 - cmdstanpy - INFO - Chain [1] start processing
15:04:20 - cmdstanpy - INFO - Chain [1] done processing


SyntaxError: 'return' outside function (21666643.py, line 45)

In [54]:
test_df_2
#Next time: make sure the final prediction is made on the basis of all files, not just create and overwrite all prediction dfs

,Country,ds,y_tr,y_agg_tr,delta_val,y_tst,yhat,level_1,test_delta_pred,final_prediction
0,Afghanistan,1950-01-01,27.7,36.4,-8.7,NaN,NaN,NaN,NaN,NaN
1,Angola,1950-01-01,36.3,36.4,-0.1,NaN,NaN,NaN,NaN,NaN
2,Benin,1950-01-01,35.0,36.4,-1.4,NaN,NaN,NaN,NaN,NaN
3,Bhutan,1950-01-01,30.1,36.4,-6.3,NaN,NaN,NaN,NaN,NaN
4,Burkina Faso,1950-01-01,33.1,36.4,-3.3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3235,Togo,2021-01-01,NaN,NaN,NaN,61.6,58.041653,16.0,0.116718,58.158371
3236,Uganda,2021-01-01,NaN,NaN,NaN,62.7,58.041653,16.0,-11.668531,46.373122
3237,Western Sahara,2021-01-01,NaN,NaN,NaN,70.8,58.041653,16.0,31.495871,89.537524
3238,Yemen,2021-01-01,NaN,NaN,NaN,63.8,58.041653,16.0,21.927893,79.969546
